# Are You Ready for the Future of AI? OpenAI's Swarm Could Change Everything
#### Learn how to configure and personalize OpenAI's Swarm framework to create powerful, collaborative multi-agent systems that meet your unique needs and drive smarter automation

Imagine a world where complex problems are solved not by a single AI, but by a team of intelligent agents working together seamlessly. OpenAI's Swarm makes this vision a reality. It's a groundbreaking framework that leverages the power of LLMs to create a collaborative system of specialized agents, each with a unique role.

Whether you're automating a time-consuming workflow, tackling a multifaceted research project, or providing exceptional customer support, Swarm enables you to deploy multiple agents, each focused on a specific task. These agents communicate effortlessly, working in perfect harmony to deliver faster, smarter, and more reliable results, much like a well-coordinated orchestra.

At its core, OpenAI's Swarm empowers you to build systems where agents act like experts in a team. They share knowledge, solve intricate problems, and adapt to challenges, all while ensuring accuracy and efficiency. The result? A multi-agent setup that revolutionizes how we approach automation, innovation, and collaboration.
In this blog, we'll take you through the foundational ideas behind OpenAI's Swarm, and implement a multi-agent system as shown below. In short, depending on the query the central agent decides which agent is best suited to handle the query.

In [ ]:
!pip install git+https://github.com/openai/swarm.git


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
# !pip install -q langchain langchain-chroma langchain-openai langchain-community pypdf sentence-transformers

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("/content/Data", show_progress=True)
docs = loader.load()
print(f"Loaded {len(docs)} documents from the folder.")

100%|██████████| 2/2 [00:08<00:00,  4.11s/it]

Loaded 2 documents from the folder.


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(f"Split the documents into {len(splits)} chunks.")

Split the documents into 19 chunks.


In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_chroma import Chroma

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
collection_name = "my_collection"

vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)
print("Vector store created and persisted to './chroma_db'")


<ipython-input-8-2fa1ec9e0188>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Vector store created and persisted to './chroma_db'


We will generate the embedding using all-MiniLM-L6-v2 model using SentenceTransformerEmbeddings method and save the embeddings in the vector database, i.e., Chroma

In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_chroma import Chroma

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
collection_name = "my_collection"

vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)
retriever = vectorstore.as_retriever()
print("Vector store created and persisted to './chroma_db'")


Vector store created and persisted to './chroma_db'


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

def retrieve_and_generate_politics(question):
    print("Calling retrieve_and_generate_politics")
    template = """Answer the question based only on the following context:
    {context}
    Question: {question}
    Answer: """

    prompt = ChatPromptTemplate.from_template(template)

    def docs2str(docs):
        if not docs:
            print("No documents retrieved!")
        else:
            print("Retrieved documents:", [doc.page_content for doc in docs])
        return "\n\n".join(doc.page_content for doc in docs)


    rag_chain = (
        {"context": retriever | docs2str, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    response = rag_chain.invoke(question)
    return response


In [ ]:
def retrieve_and_generate_sports(question):
    print("Calling retrieve_and_generate_sports")
    template = """Answer the question based only on the following context:
    {context}
    Question: {question}
    Answer: """

    prompt = ChatPromptTemplate.from_template(template)

    def docs2str(docs):
        if not docs:
            print("No documents retrieved!")
        else:
            print("Retrieved documents:", [doc.page_content for doc in docs])
        return "\n\n".join(doc.page_content for doc in docs)

    rag_chain = (
        {"context": retriever | docs2str, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    response = rag_chain.invoke(question)
    return response


Now that we have two agents set up, we need a central agent to manage them. This central agent, called the "central_agent," will determine which agents should be triggered based on the query. We will also configure the process of handing off tasks between the agents

In [ ]:
from swarm import Swarm, Agent

# Define the Politics and sports agents
politcs_agent = Agent(
    name="Politics Agent",
    instructions="You retrieve relevant information from the Politics knowledge base and generate responses to general queries about politics.",
    functions=[retrieve_and_generate_politics]
)

sports_agent = Agent(
    name="Sports Agent",
    instructions="You retrieve relevant information from the Sports knowledge base and generate responses to general queries about the sports.",
    functions=[retrieve_and_generate_sports]
)

# Define the Central Agent
central_agent = Agent(
    name="Central Agent",
    instructions="Determine if the query is about poltics or sports, and route the query accordingly."
)

# Define handoff functions
def transfer_to_politics():
    print("Handing off to the politics Agent.")
    """Transfer the task to the politcs_agent Agent for poltical queries."""
    return politcs_agent

def transfer_to_sports():
    print("Handing off to the sports agent.")
    """Transfer the task to the sport_Agent for sports queries."""
    return sports_agent

# Attach the handoff functions to the central agent
central_agent.functions = [transfer_to_politics, transfer_to_sports]


In [ ]:
client = Swarm()

# Example 1: Asking about the politics
print("\n--- Example 1: Asking about the politics ---")
messages = [{"role": "user", "content": "What is the political discussion about?"}]
response = client.run(agent=central_agent, messages=messages)
if isinstance(response, Agent):
    selected_agent = response
    result = selected_agent.functions
    print(result)
else:
    print(response.messages[-1]["content"])





--- Example 1: Asking about the politics ---
Handing off to the politics Agent.
Calling retrieve_and_generate_politics
Retrieved documents: ['India held the 2023 summit in September 2023.[40] The presidency\'s theme was Vasudhaiva Kutumbakam (Sanskrit: वसुधैव कुटुम्बकम्; English:"One Earth, One Family, One Future"[d]).[41][42] In an interview on 26 August 2023, Prime Minister Narendra Modi expressed optimism about the G20 countries\' evolving agenda under India\'s presidency, shifting toward a human-centric development approach that aligns with the concerns of the Global South, including addressing climate change, debt restructuring through the G20\'s Common Framework for debt, and a strategy for regulation of global cryptocurrencies. G20 expanded by the inclusion of African Union, it is also the first inclusion since 1999.[43][44][45]\n\nThe Brazilian presidency launched the G20 Social, a place where for the first time the organization will bring the civil society into the debate whe

In [ ]:
# Example 2: Asking about the sports
print("\n--- Example 2: Asking from the sports ---")
messages = [{"role": "user", "content": "How many test matches are to be played before the World Test Championship (WTC) and what is the maximum points that newzealand can have?"}]
response = client.run(agent=central_agent, messages=messages)
if isinstance(response, Agent):
    selected_agent = response
    result = selected_agent.functions
    print(result)
else:
    print(response.messages[-1]["content"])


--- Example 2: Asking from the sports ---
Handing off to the sports agent.
Calling retrieve_and_generate_sports
Retrieved documents: ["India\n\nPercent: 61.11, matches remaining: Aus (4 away)\n\nIndia's emphatic win in Perth takes them back to the top of the WTC points table, and keeps their chances of making it to the final at Lord's next year very much alive. To be certain of finishing among the top two without depending on other results, India still need to beat Australia 4-0: four wins and a draw would lift India to 65.79, which would be marginally more than New Zealand's maximum (64.29) if they were to blank England 3-0 at home. India would then at worst be second on the points table, after South Africa, who can finish on a maximum of 69.44 with 2-0 home wins against Sri Lanka and Pakistan.\n\nAustralia\n\nPercent: 57.69, matches remaining: Ind (4 home Tests), SL (2 away)", "India\n\nPercent: 61.11, matches remaining: Aus (4 away)\n\nIndia's emphatic win in Perth takes them back 

In [ ]:
# Example 1: Asking about the random about that neither politics not sports
print("\n--- Example 2: Asking from the sports ---")
messages = [{"role": "user", "content": "What is glycemic index and what influences the glycemic index of a food"}]
response = client.run(agent=central_agent, messages=messages)
if isinstance(response, Agent):
    selected_agent = response
    result = selected_agent.functions
    print(result)
else:
    print(response.messages[-1]["content"])


--- Example 2: Asking from the sports ---
It seems your query is neither related to politics nor sports. The glycemic index (GI) is a system that ranks foods on a scale from 0 to 100 based on how much they raise blood glucose levels after eating. Foods are classified as low (55 or less), medium (56 to 69), or high (70 or more) GI. Several factors influence the glycemic index of food:

1. **Type of Carbohydrates:** Simple carbohydrates generally have a higher GI compared to complex carbohydrates.
  
2. **Fiber Content:** Foods high in fiber typically have a lower GI because fiber slows the digestion and absorption of carbohydrates.

3. **Ripeness and Storage:** The riper a fruit or vegetable, the higher its GI because its sugars become more readily digestible.

4. **Processing and Preparation:** Foods that are more processed or ground into flour usually have a higher GI. Cooking methods can also affect GI; for example, boiling can lower the GI compared to baking or frying.

5. **Protei

# Conclusion
In conclusion, OpenAI's Swarm opens up exciting possibilities by harnessing the power of multiple intelligent agents, each equipped with specialized skills to tackle complex tasks. Throughout this blog, we've explored the core concepts of Swarm, its role in enhancing automation, and how you can set up your own multi-agent system using Large Language Models (LLMs). The real magic lies in the way these agents collaborate, boosting efficiency and scalability in various applications.
While we've covered the fundamentals, there's still much more to explore. For instance, delving deeper into advanced techniques for optimizing agent communication or integrating Swarm with other AI frameworks could elevate your projects even further. Additionally, experimenting with different types of tasks and fine-tuning agent roles will provide a more tailored solution for specific use cases.